#**Installing required libraries**

In [ ]:
!pip install datasets

#**Importing the BERT Tokenizer**

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

#**Loading the in-built squad-dataset from tf.datasets**

In [ ]:
from datasets import load_dataset
squad = load_dataset("squad", split="train[:5000]")

squad = squad.train_test_split(test_size=0.2)
squad["train"][1]

##**Definition to pre-process the training data**

In [ ]:
# We need the input as follows ==> [<START> "Question here..." <SEP> "Context here..." <END>]
# And the output should be like ==> [{start-of-answer: <INT>, end-of-answer: <INT>}]

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]

    # Tokenizing the question and context
    # Pointers:
    # 1. max_length: To restrict the "question + context" length to 384
    # 2. truncation: To truncate from the "context" part off the token if length exceeds 384
    # 3. return_offsets_mapping: offset_mapping is ==> String: "This is a sentence" --> Token: [<START>, 0, 1, 2, 3, <END>] --> offset_list: [(0, 0), (0, 4), (5, 7), (8, 9), (10, 17), (18, 18)]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    # The code below is to store the start and end positions of the answers
    ############################################################################
    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    ############################################################################

    return inputs

#**Using 'map' to apply the definition made above to every training data**

In [ ]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

#**Importing the 'model' and Preparing the training and validation data**

In [ ]:
# Importing the model from HuggingFace Transformers module
from transformers import TFAutoModelForQuestionAnswering
model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator(return_tensors="tf")

tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

tf_train_set

#**Creating the optimizer before training**

In [ ]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_squad["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

#**Compiling the model created**

In [ ]:
import tensorflow as tf
model.compile(optimizer=optimizer)

#**Training**

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=1)

#**Validating the output**

In [ ]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."
inputs = tokenizer(question, context, return_tensors="tf")
outputs = model(**inputs)
answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)